# Profils d'applications rentables pour l'App Store et les marchés de Google Play

Notre objectif dans ce projet est de trouver des profils d'applications mobiles rentables pour les marchés App Store et Google Play. Nous travaillons en tant qu'analystes de données pour une entreprise qui crée des applications mobiles Android et iOS, et notre travail consiste à permettre à notre équipe de développeurs de prendre des décisions basées sur les données en ce qui concerne le type d'applications qu'elles créent.

Dans notre entreprise, nous ne développons que des applications gratuites à télécharger et à installer, et notre principale source de revenus est constituée d'annonces intégrées aux applications. Cela signifie que nos revenus pour une application donnée sont principalement influencés par le nombre d'utilisateurs qui utilisent notre application. Notre objectif pour ce projet est d'analyser les données pour aider nos développeurs à comprendre quels types d'applications sont susceptibles d'attirer plus d'utilisateurs.

In [1]:
from csv import reader

### L'ensemble de données App Store ###
opened_file = open('Onedrive\Documents\my_datasets\AppleStore.csv', encoding="utf8")
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

### L'ensemble de données Google Play ###
opened_file = open('Onedrive\Documents\my_datasets\googleplaystore.csv', encoding="utf8")
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]


Pour faciliter l'exploration des deux ensembles de données, nous allons d'abord écrire une fonction nommée explore_data () que nous pouvons utiliser à plusieurs reprises pour explorer les lignes de manière plus lisible. Nous ajouterons également une option pour notre fonction pour afficher le nombre de lignes et de colonnes pour tout ensemble de données.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
print(android_header)
print('\n')
explore_data(android,0,2,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


Nous voyons que l'ensemble de données ***Google Play*** contient ***10841*** applications et 13 colonnes. En un coup d'œil, les colonnes qui pourraient être utiles aux fins de notre analyse sont ***«App», «Category», «Reviews», «Installs», «Type», «Price» et «Genres».***

Now let's take a look at the App Store data set.

In [4]:
print(ios_header)
print('\n')
explore_data(ios,0,2,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7197
Number of columns: 16


Nous avons ***7197*** applications ***iOS*** dans cet ensemble de données, et les colonnes qui semblent intéressantes sont: ***«track_name», «currency», «price», «rating_count_tot», «rating_count_ver» et «prime_genre».***

# Suppression de données erronées

L'ensemble de données Google Play a une section de [discussion dédiée](https://www.kaggle.com/lava18/google-play-store-apps/discussion), et nous pouvons voir que [l'une des discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) décrit une erreur pour la ligne 10472. Imprimons cette ligne et comparons-la à l'en-tête et à une autre ligne correcte.

In [5]:
print(android_header)  # header
print('\n')
print(android[10472])
print('\n')
print(android[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


La ligne 10472 correspond à l'application ***Life Made WI-Fi Touchscreen Photo Frame***, et nous pouvons voir que la note est 19. Ceci est clairement faux car la note maximale pour une application Google Play est de 5, ce problème est causé par une valeur manquante dans la colonne "Category". En conséquence, nous supprimerons cette ligne.

In [6]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


In [7]:
print(android[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


# Suppression des entrées en double

## Première partie:

Si nous examinons l'ensemble de données de ***Google Play*** assez longtemps, nous allons constater que certaines applications ont plus d'une entrée. Par exemple, l'application Instagram a quatre entrées:

In [8]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [9]:
unique_apps = []
duplicate_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

In [10]:
print('Number of duplicate apps:', len(duplicate_apps))
print('Number of unique apps:', len(unique_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:5])

Number of duplicate apps: 1181
Number of unique apps: 9659


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


En tout, il y a ***1181*** cas où une application se produit plus d'une fois.

Nous ne voulons pas compter certaines applications plus d'une fois lorsque nous analysons les données, nous devons donc supprimer les entrées en double et ne conserver qu'une seule entrée par application. Une chose que nous pourrions faire est de supprimer les lignes en double au hasard, mais nous pourrions probablement trouver un meilleur moyen.

Si vous examinez les lignes que nous avons imprimées pour l'application Instagram, la principale différence se produit à la quatrième position de chaque ligne, qui correspond au nombre d'avis. Les différents chiffres montrent que les données ont été collectées à des moments différents. Nous pouvons l'utiliser pour créer un critère de conservation des lignes. Nous ne supprimerons pas les lignes au hasard, mais nous conserverons plutôt les lignes qui ont le plus grand nombre d'avis, car plus le nombre d'avis est élevé, plus les notes sont fiables.

Pour ce faire, nous allons:

- Créez un dictionnaire où chaque clé est un nom d'application unique et la valeur est le plus grand nombre d'avis sur cette application.
- Utilisez le dictionnaire pour créer un nouvel ensemble de données, qui n'aura qu'une seule entrée par application (et nous ne sélectionnons que les applications avec le plus grand nombre d'avis)

## Deuxième Partie:

Commençons par construire le dictionnaire

In [11]:
reviews_max = {}

for row in android:
    name = row[0]
    n_reviews = float(row[3])
    
    if name in reviews_max and n_reviews > reviews_max[name]:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews


In [12]:
print('reviews_max: ', len(reviews_max))
print('unique_apps: ', len(unique_apps))

reviews_max:  9659
unique_apps:  9659


Maintenant, utilisons le dictionnaire reviews_max pour supprimer les doublons. Nous ne conserverons que les entrées avec le plus grand nombre d'avis. Dans la cellule de code ci-dessous:


In [13]:
android_clean = []
already_added = []

for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(row)
        already_added.append(name)

In [14]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


We have 9659 rows, just as expected

# Suppression des applications non anglaises


Si vous explorez suffisamment les ensembles de données, vous remarquerez que les noms de certaines applications suggèrent qu'elles ne sont pas dirigées vers un public anglophone. Ci-dessous, nous voyons quelques exemples des deux ensembles de données:

In [15]:
def is_english(string):
    for letter in string:
        if ord(letter) > 127:
            return False          
    return True

In [16]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(ord('™'))
print(ord('😜'))

True
False
False
False
8482
128540


Pour minimiser l'impact de la perte de données, nous ne supprimerons une application que si son nom comporte plus de trois caractères non ASCII:

In [17]:
def is_english(string):
    n=0
    for letter in string:
        if ord(letter) > 127:
            n += 1
    if n > 3:
        return False
    return True

In [18]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


In [19]:
english_android = []
english_ios = []

for row in android_clean:
    name = row[0]
    if is_english(name):
        english_android.append(row)
        
for row in ios:
    name = row[1]
    if is_english(name):
        english_ios.append(row)

In [21]:
explore_data(english_android, 0, 3, True)
print('\n')
explore_data(english_ios, 0, 3, True)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

#### Nous pouvons voir qu'il nous reste 9614 applications Android et 6183 applications iOS.

# Isoler les applications gratuites

Comme nous l'avons mentionné dans l'introduction, nous ne développons que des applications qui sont gratuites à télécharger et à installer, et notre principale source de revenus est constituée d'annonces intégrées à l'application. Nos ensembles de données contiennent à la fois des applications gratuites et non gratuites, et nous devrons isoler uniquement les applications gratuites pour notre analyse. Ci-dessous, nous isolons les applications gratuites pour nos deux ensembles de données.

In [27]:
free_android = []
free_ios = []

for row in english_android:
    price = row[7]
    if price == '0':
        free_android.append(row)

for row in english_ios:
    price = row[4]
    if price == '0.0':
        free_ios.append(row)

In [42]:
explore_data(free_android, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 8864
Number of columns: 13


In [43]:
explore_data(free_ios,0, 5, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 3222
Number of columns: 16


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

# Applications les plus courantes par genre


Jusqu'à présent, nous avons passé beaucoup de temps à nettoyer les données et:

 -    Suppression des données inexactes
 -    Suppression des entrées d'application en double
 -    Suppression des applications non anglaises
 -   Isolé les applications gratuites
     
     
Comme nous l'avons mentionné dans l'introduction, notre objectif est de déterminer les types d'applications susceptibles d'attirer plus d'utilisateurs, car nos revenus sont fortement influencés par le nombre de personnes utilisant nos applications.

Étant donné que notre objectif final est d'ajouter l'application à la fois sur l'App Store et sur Google Play, nous devons trouver des profils d'application qui réussissent sur les deux marchés. Par exemple, un profil qui pourrait bien fonctionner pour les deux marchés pourrait être une application de productivité qui utilise la gamification.

Commençons l'analyse en nous faisant une idée des genres les plus courants pour chaque marché. Pour cela, nous allons créer un tableau de fréquences pour la colonne prime_genre de l'ensemble de données App Store et les colonnes Genres et Category de l'ensemble de données Google Play.

Nous allons créer deux fonctions que nous pouvons utiliser pour analyser les tables de fréquences:

- Une fonction pour générer des tableaux de fréquences affichant des pourcentages
- Une autre fonction que nous pouvons utiliser pour afficher les pourcentages dans un ordre décroissant

In [79]:
def freq_table(dataset, index):
    freq_dict = {}
    for row in dataset:
        value = row[index]
        if value in freq_dict:
            freq_dict[value] += 1
        else:
            freq_dict[value] = 1
    for key in freq_dict:
        freq_dict[key]= freq_dict[key]/len(dataset) * 100
    return freq_dict
    

In [81]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [85]:
display_table(free_ios, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


On peut voir que parmi les applications gratuites en anglais, plus de la moitié (58,16%) sont des jeux. Les applications de divertissement sont proches de 8%, suivies des applications photo et vidéo, qui sont proches de 5%. Seulement 3,66% des applications sont conçues pour l'éducation, suivies des applications de réseaux sociaux qui représentent 3,29% des applications de notre ensemble de données.

L'impression générale est que l'App Store (au moins la partie contenant des applications gratuites en anglais) est dominé par des applications conçues pour le plaisir (jeux, divertissement, photo et vidéo, réseaux sociaux, sports, musique, etc.), tandis que des applications avec des applications pratiques les objectifs (éducation, achats, services publics, productivité, style de vie, etc.) sont plus rares. Cependant, le fait que les applications amusantes soient les plus nombreuses ne signifie pas non plus qu'elles ont également le plus grand nombre d'utilisateurs - la demande peut ne pas être la même que l'offre.

In [87]:
display_table(free_android, 1)  #Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

Le paysage semble sensiblement différent sur Google Play: il n'y a pas beaucoup d'applications conçues pour le plaisir, et il semble qu'un bon nombre d'applications soient conçues à des fins pratiques (famille, outils, entreprise, style de vie, productivité, etc.). Cependant, si nous approfondissons cette question, nous pouvons voir que la catégorie famille (qui représente près de 19% des applications) signifie principalement des jeux pour enfants.

![Image](https://camo.githubusercontent.com/9bf24b9efc3d88a3d55f5c09e314987941f0bab5/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f64712d636f6e74656e742f3335302f7079316d385f66616d696c792e706e67)

Même ainsi, les applications pratiques semblent avoir une meilleure représentation sur Google Play par rapport à l'App Store. Cette image est également confirmée par le tableau des fréquences que nous voyons pour la colonne Genres:

In [84]:
display_table(free_android, 9)  #Genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


La différence entre les colonnes Genres et Category n'est pas claire, mais une chose que nous pouvons remarquer est que la colonne Genres est beaucoup plus granulaire (elle comporte plus de catégories). Nous cherchons uniquement une vue d'ensemble pour le moment, nous allons donc travailler avec la colonne Catégorie à l'avenir.

Jusqu'à présent, nous avons constaté que l'App Store est dominé par des applications conçues pour le plaisir, tandis que Google Play présente un paysage plus équilibré d'applications pratiques et amusantes. Nous aimerions maintenant avoir une idée du type d'applications qui ont le plus d'utilisateurs.


# Applications les plus populaires par genre sur l'App Store

Une façon de savoir quels genres sont les plus populaires (qui ont le plus d'utilisateurs) consiste à calculer le nombre moyen d'installations pour chaque genre d'application. Pour l'ensemble de données Google Play, nous pouvons trouver ces informations dans la colonne Installations, mais pour l'ensemble de données App Store, ces informations sont manquantes. Pour contourner ce problème, nous prendrons le nombre total de notes d'utilisateurs comme proxy, que nous pouvons trouver dans l'application rating_count_tot.

Ci-dessous, nous calculons le nombre moyen d'avis d'utilisateurs par genre d'application sur l'App Store:

In [94]:
for genre in freq_table(free_ios, 11):
    total = 0
    len_genre = 0
    for row in free_ios:
        genre_app = row[11]
        if genre_app == genre:
            rate = float(row[5])
            total += rate
            len_genre += 1
    avg_user_rating = total / len_genre
    print(genre, ': ', avg_user_rating)

Social Networking :  71548.34905660378
Photo & Video :  28441.54375
Games :  22788.6696905016
Music :  57326.530303030304
Reference :  74942.11111111111
Health & Fitness :  23298.015384615384
Weather :  52279.892857142855
Utilities :  18684.456790123455
Travel :  28243.8
Shopping :  26919.690476190477
News :  21248.023255813954
Navigation :  86090.33333333333
Lifestyle :  16485.764705882353
Entertainment :  14029.830708661417
Food & Drink :  33333.92307692308
Sports :  23008.898550724636
Book :  39758.5
Finance :  31467.944444444445
Education :  7003.983050847458
Productivity :  21028.410714285714
Business :  7491.117647058823
Catalogs :  4004.0
Medical :  612.0


En moyenne, les applications de navigation ont le plus grand nombre d'avis d'utilisateurs, mais ce chiffre est fortement influencé par Waze et Google Maps, qui comptent près d'un demi-million d'avis d'utilisateurs ensemble:

In [97]:
for app in free_ios:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Le même schéma s'applique aux applications de réseaux sociaux, où le nombre moyen est fortement influencé par quelques géants comme Facebook, Pinterest, Skype, etc. Il en va de même pour les applications de musique, où quelques grands acteurs comme Pandora, Spotify et Shazam influencent fortement le numéro médiane.

Notre objectif est de trouver des genres populaires, mais la navigation, les réseaux sociaux ou les applications musicales peuvent sembler plus populaires qu'ils ne le sont en réalité. Le nombre moyen de notes semble faussé par très peu d'applications qui ont des centaines de milliers de notes d'utilisateurs, tandis que les autres applications peuvent avoir du mal à dépasser le seuil de 10000. Nous pourrions obtenir une meilleure image en supprimant ces applications extrêmement populaires pour chaque genre, puis en retravaillant les moyennes, mais nous laisserons ce niveau de détail pour plus tard.

Les applications de référence ont en moyenne 74942 évaluations d'utilisateurs, mais ce sont en fait la Bible et Dictionary.com qui faussent la note moyenne:

In [98]:
for app in free_ios:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Cependant, ce créneau semble montrer un certain potentiel. Une chose que nous pourrions faire est de prendre un autre livre populaire et de le transformer en une application où nous pourrions ajouter différentes fonctionnalités en plus de la version brute du livre. Cela peut inclure des citations quotidiennes du livre, une version audio du livre, des quiz sur le livre, etc. En plus de cela, nous pouvons également intégrer un dictionnaire dans l'application, afin que les utilisateurs n'aient pas besoin de quitter notre application pour regarder des mots dans une application externe.

Cette idée semble bien cadrer avec le fait que l'App Store est dominé par les applications pour le plaisir. Cela suggère que le marché pourrait être un peu saturé d'applications pour le plaisir, ce qui signifie qu'une application pratique pourrait avoir plus de chances de se démarquer parmi le grand nombre d'applications sur l'App Store.

D'autres genres qui semblent populaires incluent la météo, le livre, la nourriture et les boissons ou la finance. Le genre de livre semble chevaucher un peu l'idée d'application que nous avons décrite ci-dessus, mais les autres genres ne nous semblent pas trop intéressants:

 -   Applications météo - les gens ne passent généralement pas trop de temps dans l'application, et les chances de tirer profit des ajouts intégrés à l'application sont faibles. En outre, obtenir des données météorologiques en direct fiables peut nous obliger à connecter nos applications à des API non libres.

 -    Nourriture et boisson - des exemples ici incluent Starbucks, Dunkin 'Donuts, McDonald's, etc. La création d'une application populaire de nourriture et de boisson nécessite donc une cuisine et un service de livraison, ce qui est en dehors du champ d'action de notre société.

 -    Applications financières - ces applications impliquent des opérations bancaires, le paiement de factures, le transfert d'argent, etc. La création d'une application financière nécessite une connaissance du domaine, et nous ne voulons pas embaucher un expert en finance uniquement pour créer une application.
 
 
Analysons maintenant un peu le marché de Google Play.

# Applications les plus populaires par genre sur Google Play

Pour le marché de Google Play, nous avons en fait des données sur le nombre d'installations, nous devrions donc être en mesure d'avoir une image plus claire de la popularité du genre. Cependant, les numéros d'installation ne semblent pas assez précis - nous pouvons voir que la plupart des valeurs sont ouvertes (100+, 1,000+, 5,000+, etc.):

In [100]:
for category in freq_table(free_android, 1):
    total = 0
    len_category = 0
    for row in free_android:
        category_app = row[1]
        if category_app == category:
            n_install = row[5]
            n_install = n_install.replace('+', '')
            n_install = n_install.replace(',', '')
            n_install = float(n_install)
            total += n_install
            len_category += 1
    avg_number_install = total / len_category
    print(category, ': ', avg_number_install)

ART_AND_DESIGN :  1986335.0877192982
AUTO_AND_VEHICLES :  647317.8170731707
BEAUTY :  513151.88679245283
BOOKS_AND_REFERENCE :  8767811.894736841
BUSINESS :  1712290.1474201474
COMICS :  817657.2727272727
COMMUNICATION :  38456119.167247385
DATING :  854028.8303030303
EDUCATION :  1833495.145631068
ENTERTAINMENT :  11640705.88235294
EVENTS :  253542.22222222222
FINANCE :  1387692.475609756
FOOD_AND_DRINK :  1924897.7363636363
HEALTH_AND_FITNESS :  4188821.9853479853
HOUSE_AND_HOME :  1331540.5616438356
LIBRARIES_AND_DEMO :  638503.734939759
LIFESTYLE :  1437816.2687861272
GAME :  15588015.603248259
FAMILY :  3695641.8198090694
MEDICAL :  120550.61980830671
SOCIAL :  23253652.127118643
SHOPPING :  7036877.311557789
PHOTOGRAPHY :  17840110.40229885
SPORTS :  3638640.1428571427
TRAVEL_AND_LOCAL :  13984077.710144928
TOOLS :  10801391.298666667
PERSONALIZATION :  5201482.6122448975
PRODUCTIVITY :  16787331.344927534
PARENTING :  542603.6206896552
WEATHER :  5074486.197183099
VIDEO_PLAYERS 

En moyenne, les applications de communication sont les plus installées: 38 456 119. Ce nombre est fortement biaisé par quelques applications qui comptent plus d'un milliard d'installations (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail et Hangouts), et quelques autres avec plus de 100 et 500 millions d'installations:

In [107]:
for app in free_android:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Si nous supprimions toutes les applications de communication qui ont plus de 100 millions d'installations, la moyenne serait réduite d'environ dix fois:

In [108]:
under_100_m = []

for app in free_android:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

Nous constatons le même schéma pour la catégorie des lecteurs vidéo, qui est le finaliste avec 24 727 872 installations. Le marché est dominé par des applications telles que Youtube, Google Play Movies & TV ou MX Player. Le modèle est répété pour les applications sociales (où nous avons des géants comme Facebook, Instagram, Google+, etc.), les applications de photographie (Google Photos et autres éditeurs de photos populaires) ou les applications de productivité (Microsoft Word, Dropbox, Google Calendar, Evernote, etc. .).

Encore une fois, la principale préoccupation est que ces genres d'applications peuvent sembler plus populaires qu'ils ne le sont en réalité. De plus, ces niches semblent être dominées par quelques géants difficiles à concurrencer.

Le genre de jeu semble assez populaire, mais nous avons précédemment découvert que cette partie du marché semble un peu saturée, nous aimerions donc proposer une recommandation d'application différente si possible.

Le genre de livres et de référence semble également assez populaire, avec un nombre moyen d'installations de 8 767 811. Il est intéressant d'explorer cela plus en profondeur, car nous avons trouvé que ce genre a un certain potentiel pour bien fonctionner sur l'App Store, et notre objectif est de recommander un genre d'application qui présente un potentiel de rentabilité à la fois sur l'App Store et sur Google Play.

Jetons un coup d'œil à certaines des applications de ce genre et à leur nombre d'installations:

In [110]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

'BOOKS_AND_REFERENCE' comprend une variété d'applications: des logiciels de traitement et de lecture de livres électroniques, diverses collections de bibliothèques, de dictionnaires, de tutoriels sur la programmation ou les langages, etc. Il semble qu'il y ait encore un petit nombre d'applications extrêmement populaires qui faussent la moyenne:

In [111]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


Cependant, il semble qu'il n'y ait que quelques applications très populaires, donc ce marché montre encore du potentiel. Essayons de trouver des idées d'applications basées sur le type d'applications qui se situent quelque part au milieu en termes de popularité (entre 1,000,000 et 100,000,000 téléchargements):

In [112]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

Ce créneau semble être dominé par les logiciels de traitement et de lecture de livres électroniques, ainsi que par diverses collections de bibliothèques et de dictionnaires, donc ce n'est probablement pas une bonne idée de créer des applications similaires car il y aura une concurrence importante.

Nous remarquons également qu'il existe de nombreuses applications construites autour du livre Coran, ce qui suggère que la création d'une application autour d'un livre populaire peut être rentable. Il semble que prendre un livre populaire (peut-être un livre plus récent) et le transformer en une application pourrait être rentable pour les marchés de Google Play et de l'App Store.

Cependant, il semble que le marché soit déjà plein de bibliothèques, nous devons donc ajouter des fonctionnalités spéciales en plus de la version brute du livre. Cela peut inclure des citations quotidiennes du livre, une version audio du livre, des quiz sur le livre, un forum où les gens peuvent discuter du livre, etc.

# Conclusions

Dans ce projet, nous avons analysé les données sur l'App Store et les applications mobiles Google Play dans le but de recommander un profil d'application qui peut être rentable pour les deux marchés.

Nous avons conclu que prendre un livre populaire (peut-être un livre plus récent) et le transformer en une application pourrait être rentable pour les marchés de Google Play et de l'App Store. Les marchés sont déjà pleins de bibliothèques, nous devons donc ajouter des fonctionnalités spéciales en plus de la version brute du livre. Cela peut inclure des citations quotidiennes du livre, une version audio du livre, des quiz sur le livre, un forum où les gens peuvent discuter du livre, etc.